# 9장: 복잡한 프롬프트 처음부터 만들기

- [수업](#lesson)
- [연습 문제](#exercises)

## 설정

다음 설정 셀을 실행하여 API 키를 로드하고 `get_completion` 도우미 함수를 설정하세요.

In [ ]:
!pip install anthropic

# Python의 내장 정규 표현식 라이브러리 가져오기
import re
import anthropic

# IPython 저장소에서 API_KEY 및 MODEL_NAME 변수 검색
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## 수업

마지막 장까지 오신 것을 축하합니다! 이제 모든 것을 종합하고 **독특하고 복잡한 프롬프트를 만드는 방법**을 배울 시간입니다.

아래에서는 **복잡한 프롬프트에 권장하는 가이드 구조**를 사용할 것입니다. 이 장의 후반부에서는 산업별 특화된 프롬프트 몇 가지를 보여드리고 이러한 프롬프트가 어떻게 유사하게 구조화되어 있는지 설명하겠습니다.

**참고:** **모든 프롬프트가 다음과 같은 복잡한 구조의 모든 요소를 필요로 하는 것은 아닙니다**. 요소를 포함하거나 제외하면서 실험해보고 Claude의 응답에 어떤 영향을 미치는지 확인해보세요. 일반적으로 **프롬프트를 먼저 작동시키기 위해 많은 프롬프트 요소를 사용한 다음, 나중에 프롬프트를 정제하고 간소화하는 것이 좋습니다**.

### 복잡한 프롬프트 구조

다음은 복잡한 프롬프트를 구성하는 데 사용할 수 있는 10가지 주요 요소입니다:

1. **`user` 역할**: 메시지 배열에서 항상 `user` 역할로 시작하는지 확인하세요.

2. **작업 컨텍스트**: Claude에게 맡아야 할 역할이나 프롬프트로 수행하려는 목표와 전반적인 작업에 대한 컨텍스트를 제공합니다.

3. **어조 컨텍스트**: 상호작용에 중요한 경우, Claude에게 사용해야 할 어조를 알려주세요.

4. **상세한 작업 설명 및 규칙**: Claude가 수행해야 할 특정 작업과 Claude가 따라야 할 규칙을 확장합니다.

5. **예제**: Claude에게 모방할 수 있는 이상적인 응답의 예를 제공하세요. XML 태그로 감싸세요.

6. **처리할 입력 데이터**: 프롬프트 내에서 Claude가 처리해야 하는 데이터가 있는 경우, 여기에 관련 XML 태그 내에 포함하세요.

7. **즉각적인 작업 설명 또는 요청**: Claude에게 프롬프트의 작업을 수행하기 위해 즉시 수행해야 할 것을 알려주세요.

8. **예지력 (단계별 사고)**: 여러 단계가 있는 작업의 경우, Claude에게 답변을 제공하기 전에 단계별로 생각하도록 지시하세요.

9. **출력 형식 지정**: Claude의 응답을 특정 방식으로 형식화하려면, Claude에게 그 형식이 무엇인지 명확하게 알려주세요.

10. **Claude의 응답 미리 채우기**: Claude의 답변을 미리 채워진 단어로 시작하여 Claude의 행동이나 응답을 유도할 수 있습니다.

### 예제 - 커리어 코치 챗봇

다음 구조는 여러 프롬프트 엔지니어링 요소를 결합하며 복잡한 프롬프트의 좋은 출발점입니다. 일부 요소의 순서는 중요하지만, 다른 요소는 그렇지 않습니다. 일반적으로 이 순서를 따르면 훌륭한 프롬프트를 시작하는 데 좋은 출발점이 될 것입니다.

다음 예제에서는 Claude가 친절한 커리어 코치 역할을 하도록 프롬프트하는 것입니다.

```python
# 입력 변수
HISTORY = """Customer: 사회학 전공자를 위한 두 가지 가능한 직업을 알려주세요.

Joe: 사회학 전공자를 위한 두 가지 잠재적 직업은 다음과 같습니다:

- 사회복지사 - 사회학은 인간 행동과 사회 시스템을 이해하는 강력한 기반을 제공합니다.
- 인사 전문가 - 사회학에서 배운 그룹 역학과 조직 행동에 대한 이해는 인사 분야에 적용할 수 있습니다."""

QUESTION = "두 직업 중 어떤 것이 학사 학위 이상을 요구하나요?"

# 프롬프트 요소
TASK_CONTEXT = "당신은 AdAstra Careers 회사에서 만든 Joe라는 AI 커리어 코치 역할을 할 것입니다."
TONE_CONTEXT = "친절한 고객 서비스 어조를 유지해야 합니다."
TASK_DESCRIPTION = "항상 Joe로서 캐릭터를 유지하세요."
EXAMPLES = """<example>
Customer: 안녕하세요, 당신은 어떻게 만들어졌고 무엇을 하나요?
Joe: 안녕하세요! 제 이름은 Joe이고, AdAstra Careers에서 경력 조언을 제공하기 위해 만들어졌습니다.
</example>"""
INPUT_DATA = f"""<history>
{HISTORY}
</history>

<question>
{QUESTION}
</question>"""
IMMEDIATE_TASK = "사용자의 질문에 어떻게 응답하시겠습니까?"
PRECOGNITION = "응답하기 전에 먼저 답변에 대해 생각해보세요."
OUTPUT_FORMATTING = "응답을 <response></response> 태그 안에 넣으세요."
PREFILL = "[Joe] <response>"
```

### 예제 - 법률 서비스

**법률 직종 내의 프롬프트는 다음과 같은 이유로 꽤 복잡할 수 있습니다**:
- 긴 문서 파싱
- 복잡한 주제 다루기
- 매우 특정한 방식으로 출력 형식 지정
- 다단계 분석 프로세스 따르기

복잡한 프롬프트 템플릿을 사용하여 특정 법률 사용 사례에 대한 프롬프트를 구성하는 방법을 살펴보겠습니다.

```python
# 입력 변수
LEGAL_RESEARCH = """<search_results>
<search_result id=1>
동물 건강 산업은 지난 해 동안 여러 특허 및 상표 소송에 휘말렸습니다.
</search_result>
<search_result id=3>
2005년 미국 걸프 해안을 강타한 허리케인 카트리나의 여파로 자연 재해 동안 동물을 대하는 방식에 변화가 생겼습니다. 2006년, 하와이, 루이지애나, 뉴햄프셔는 모두 재해 시 반려동물을 위한 대피소 제공 및 서비스 동물이 그들이 봉사하는 사람들과 함께 있도록 허용하는 등 재해 동안 동물 관리에 관한 문제를 다루는 법률을 제정했습니다.
</search_result>
</search_results>"""

QUESTION = "허리케인 동안 반려동물을 어떻게 해야 하는지에 대한 법률이 있나요?"

# 프롬프트 요소
TASK_CONTEXT = "당신은 전문 변호사입니다."
INPUT_DATA = f"""<legal_research>
{LEGAL_RESEARCH}
</legal_research>"""
EXAMPLES = """<example>
법령은 10년 후에 만료됩니다. [3].
</example>"""
TASK_DESCRIPTION = f"""다음 질문에 대한 명확하고 간결한 답변을 작성하세요:
<question>
{QUESTION}
</question>"""
PRECOGNITION = "답변하기 전에 연구에서 가장 관련 있는 인용문을 <relevant_quotes> 태그 안에 추출하세요."
OUTPUT_FORMATTING = "두 단락 응답을 <answer> 태그 안에 넣으세요."
PREFILL = "<relevant_quotes>"
```

---

## 연습 문제

- [연습 문제 9.1 - 금융 서비스 챗봇](#exercise-91---financial-services-chatbot)
- [연습 문제 9.2 - 코드봇](#exercise-92---codebot)

### 연습 문제 9.1 - 금융 서비스 챗봇

금융 직종 내의 프롬프트도 법률 프롬프트와 유사한 이유로 꽤 복잡할 수 있습니다. 여기에서는 **세금 정보를 분석하고 질문에 답변하는 데** Claude를 사용하는 금융 사용 사례에 대한 연습을 제공합니다. 법률 서비스 예제와 마찬가지로, 우리의 솔루션 프롬프트가 다른 흐름으로 더 의미가 있기 때문에 몇 가지 요소의 순서를 변경했습니다(그러나 다른 구조도 작동할 것입니다).

변수 내용(`{QUESTION}` 및 `{TAX_CODE}`)을 읽어보고 Claude가 어떤 내용으로 작업할 것으로 예상되는지 이해하는 것이 좋습니다. 실제 변수 내용이 대체될 수 있도록 프롬프트 어딘가에 `{QUESTION}` 및 `{TAX_CODE}`를 직접 참조해야 합니다(다른 예제와 같이 f-string 구문 사용).

프롬프트 요소 필드를 설명과 이전 복잡한 프롬프트 예제에서 본 내용과 일치하는 내용으로 채우세요. 모든 프롬프트 요소를 채운 후 셀을 실행하여 연결된 프롬프트와 Claude의 응답을 확인하세요.

프롬프트 엔지니어링은 특히 크고 복잡한 프롬프트의 경우 순전히 공식적인 경우가 거의 없다는 점을 기억하세요! 테스트 케이스를 개발하고 **각 상황에 가장 적합한 것을 확인하기 위해 다양한 프롬프트와 프롬프트 구조를 시도하는 것이 중요합니다**. 프롬프트 요소의 순서를 변경하는 경우 `COMBINE ELEMENTS` 섹션에서 연결 순서도 변경해야 한다는 점에 유의하세요.

```python
# 입력 변수
QUESTION = "83b 선거를 하기 위해 얼마나 시간이 있나요?"

TAX_CODE = """
(b)양도 연도에 총소득에 포함하기 위한 선거
(1)일반
재산이 양도되는 것과 관련하여 서비스를 수행하는 사람은 해당 재산이 양도되는 과세 연도에 다음의 초과분을 자신의 총소득에 포함하기로 선거할 수 있습니다—
(A)양도 시점의 해당 재산의 공정 시장 가치(기간이 만료되지 않는 제한 외에는 어떤 제한도 고려하지 않고 결정됨), 초과
(B)해당 재산에 대해 지불된 금액(있는 경우).
이러한 선거가 이루어지면, 하위 섹션 (a)는 해당 재산의 양도에 적용되지 않으며, 해당 재산이 이후에 몰수되는 경우 해당 몰수에 대한 공제가 허용되지 않습니다.
(2)선거
단락 (1)에 따른 재산 양도에 관한 선거는 장관이 규정하는 방식으로 이루어져야 하며, 해당 양도일로부터 30일 이내에 이루어져야 합니다. 이러한 선거는 장관의 동의 없이는 취소할 수 없습니다.
"""

# 프롬프트 요소
TASK_CONTEXT = ""
TONE_CONTEXT = ""
INPUT_DATA = ""
EXAMPLES = ""
TASK_DESCRIPTION = ""
IMMEDIATE_TASK = ""
PRECOGNITION = ""
OUTPUT_FORMATTING = ""
PREFILL = ""
```

### 연습 문제 9.2 - 코드봇

이 연습에서는 **코드를 읽고 필요한 경우 안내 수정을 제공하는 코딩 지원 및 교육 봇**에 대한 프롬프트를 작성할 것입니다. 프롬프트 요소 필드를 설명과 이전 복잡한 프롬프트 예제에서 본 내용과 일치하는 내용으로 채우세요. 모든 프롬프트 요소를 채운 후 셀을 실행하여 연결된 프롬프트와 Claude의 응답을 확인하세요.

변수 내용(`{CODE}`)을 읽어보고 Claude가 어떤 내용으로 작업할 것으로 예상되는지 이해하는 것이 좋습니다. 실제 변수 내용이 대체될 수 있도록 프롬프트 어딘가에 `{CODE}`를 직접 참조해야 합니다(다른 예제와 같이 f-string 구문 사용).

```python
# 입력 변수
CODE = """
# 곱셈 역수를 출력하는 함수
def print_multiplicative_inverses(x, n):
  for i in range(n):
    print(x / i) 
"""

# 프롬프트 요소
TASK_CONTEXT = ""
TONE_CONTEXT = ""
TASK_DESCRIPTION = ""
EXAMPLES = ""
INPUT_DATA = ""
IMMEDIATE_TASK = ""
PRECOGNITION = ""
OUTPUT_FORMATTING = ""
PREFILL = ""
```

### 축하합니다 & 다음 단계!

모든 연습 문제를 완료했다면, **이제 당신은 LLM 전문가 중 상위 0.1%에 속합니다**. 엘리트 중 한 명입니다!

단계별 사고부터 역할 할당, 예제 사용, 일반적인 명확한 글쓰기에 이르기까지 배운 기술은 **무수히 많은 방식으로 병합, 재혼합, 적응될 수 있습니다**.

프롬프트 엔지니어링은 매우 새로운 분야이므로 열린 마음을 유지하세요. 다음 훌륭한 프롬프팅 트릭을 발견하는 사람이 바로 당신일 수 있습니다.

**더 많은 좋은 프롬프트 예제**를 보고 싶다면:
- [cookbook](https://anthropic.com/cookbook)에서 프로덕션 준비가 된 프롬프트 예제를 배우세요
- [프롬프팅 가이드](https://docs.anthropic.com/claude/docs/prompt-engineering)를 읽어보세요
- 영감을 얻기 위해 [프롬프트 라이브러리](https://anthropic.com/prompts)를 확인하세요
- Claude가 프롬프트 템플릿을 작성하도록 실험적인 [메타프롬프트](https://docs.anthropic.com/claude/docs/helper-metaprompt-experimental)를 시도해보세요!
- [디스코드 서버](https://anthropic.com/discord)에서 질문하세요
- 온도 및 `max_tokens`와 같은 [Anthropic API 매개변수](https://docs.anthropic.com/claude/reference/complete_post)에 대해 알아보세요
- 학문적인 느낌이 든다면, 프롬프트 엔지니어링에 관한 [논문](https://www.promptingguide.ai/papers)을 읽어보세요
- 관심 있는 일을 Claude가 수행하도록 프롬프트를 구축하는 연습을 하세요